Блокнот с демонстрацией некоторых возможностей по обработке естественного языка, заготовка для лабораторной работы № 5 по курсу "Методы искусственного интеллекта".

# 1. Лингвистический анализ

Лингвистический анализ - это разбор текста, выделение слов, определение морфологических характеристик, структуры предложений и пр. Лингвистический анализ текста может быть как самостоятельной целью (если вы лингвист), так и выступать в качестве вспомогательного (и предварительного) этапа для каких-то прикладных задач обработки текста.

Ознакомимся кратко с основными этапами лингвистического анализа, для чего воспользуемся библиотекой `natasha`. Эта библиотека предоставляет возможность проводит различные виды анализа - от графематического до синтаксического, обладает относительно низкими требованиями к ресурсам и достаточно высокой производительностью. Альтернативных решений несколько - одним из самых популярных моделей является библиотека Spacy. 



In [1]:
import natasha
import pymorphy2  # библиотека для морфологического анализа русского языка
                  # является одной из зависимостей natasha
import pandas as pd

Загрузим набор данных. Мы будем использовать подмножество набора новостей с сайта Lenta.ru (https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta). В наборе данных оставлены только новости на тему экономики, культуры и спорта.

"Усеченный" набор следует скачать по адресу https://disk.yandex.ru/d/PmkxMoQjN7zHCg 
и распаковать таким образом, чтобы файл `lenta-subset.csv` оказался по следующему пути: `data/lenta`


In [2]:
df = pd.read_csv('data/lenta/lenta-subset.csv')
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1999/10/04/tv/,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,Все,1999/10/04
1,https://lenta.ru/news/1999/10/04/volkswagen/,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,Все,1999/10/04
2,https://lenta.ru/news/1999/10/04/tumen/,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,Все,1999/10/04
3,https://lenta.ru/news/1999/10/05/sprint/,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,Все,1999/10/05
4,https://lenta.ru/news/1999/10/05/volga/,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,Все,1999/10/05
...,...,...,...,...,...,...
197733,https://lenta.ru/news/2018/12/15/oleinik/,Российский боец UFC включен в Книгу рекордов Г...,Российский боец смешанного стиля (MMA) Алексей...,Спорт,Бокс и ММА,2018/12/15
197734,https://lenta.ru/news/2018/12/15/frank_myr/,Бывший чемпион UFC не выдержал кровопролития и...,Американский боец смешанного стиля (MMA) Фрэн...,Спорт,Бокс и ММА,2018/12/15
197735,https://lenta.ru/news/2018/12/15/mebel/,Моуринью сравнил футболистов с мебелью,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,Спорт,Футбол,2018/12/15
197736,https://lenta.ru/news/2018/12/15/putinrap/,Путин предостерег от запретов рэп-концертов,"Президент России Владимир Путин, выступая на з...",Культура,Музыка,2018/12/15


## 1.1 Токенизация (графематический анализ)

Последовательность обработки документа с помощью библиотеки `natasha` строится вокруг концепции документ (класс `Doc`). Изначально документ представляет собой простой текст, но по мере применения различных алгоритмов лингвистического анализа он "обрастает" новыми деталями, касающимися того или иного аспекта.  

In [3]:
doc = natasha.Doc(df.iloc[0].text)

В библиотеке `natasha` за токенизацию отвечает компонент `Segmenter`. Создадим экземпляр компонента сегментации и применим его к тексту:

In [4]:
segmenter = natasha.Segmenter()
doc.segment(segmenter)

In [5]:
doc

Doc(text='С 1 января 2000 года все телеканалы будут оплачив..., tokens=[...], sents=[...])

Мы видим, что поле `text` документа по-прежнему хранит исходный текст, но помимо него у экземпляра `Doc` появились свойства `tokens` и `sents`, содержащие токены и предложения соответственно:

In [6]:
doc.sents

[DocSent(stop=104, text='С 1 января 2000 года все телеканалы будут оплачив..., tokens=[...]),
 DocSent(start=105, stop=299, text='Как сообщило министерство Российской Федерации по..., tokens=[...]),
 DocSent(start=300, stop=485, text='В настоящее время общенациональные телеканалы (ОР..., tokens=[...]),
 DocSent(start=486, stop=596, text='С 1 января для ОРТ, НТВ и ВГТРК тарифы будут увел..., tokens=[...])]

In [7]:
doc.tokens[:5]

[DocToken(stop=1, text='С'),
 DocToken(start=2, stop=3, text='1'),
 DocToken(start=4, stop=10, text='января'),
 DocToken(start=11, stop=15, text='2000'),
 DocToken(start=16, stop=20, text='года')]

Следует иметь в виду, что свойство `tokens` содержит все токены подряд (без разбивки на предложения), если необходимо получать токены по предложениям, то это можно делать через свойство `sents`:

In [8]:
doc.sents[0].tokens

[DocToken(stop=1, text='С'),
 DocToken(start=2, stop=3, text='1'),
 DocToken(start=4, stop=10, text='января'),
 DocToken(start=11, stop=15, text='2000'),
 DocToken(start=16, stop=20, text='года'),
 DocToken(start=21, stop=24, text='все'),
 DocToken(start=25, stop=35, text='телеканалы'),
 DocToken(start=36, stop=41, text='будут'),
 DocToken(start=42, stop=52, text='оплачивать'),
 DocToken(start=53, stop=59, text='услуги'),
 DocToken(start=60, stop=68, text='передачи'),
 DocToken(start=69, stop=85, text='телерадиосигнала'),
 DocToken(start=86, stop=88, text='по'),
 DocToken(start=89, stop=96, text='единому'),
 DocToken(start=97, stop=103, text='тарифу'),
 DocToken(start=103, stop=104, text='.')]

Список токенов состоит из экземпляров класса `DocToken`, если (например, при работе с каким-то внешним инструментом) нужно преобразовать его в список строк, то можно сделать это следующим образом:

In [9]:
[x.text for x in doc.sents[0].tokens]

['С',
 '1',
 'января',
 '2000',
 'года',
 'все',
 'телеканалы',
 'будут',
 'оплачивать',
 'услуги',
 'передачи',
 'телерадиосигнала',
 'по',
 'единому',
 'тарифу',
 '.']

## 1.2 Морфологический анализ

На этапе морфологического анализа токены сопровождаются морфологическими тегами (часть речи, род, падеж и пр.). При этом, по форме слова не всегда однозначно понятно к какой части речи она может относиться. Морфологический анализ связан с разрешением частеречной омонимии, для чего используется контекст слова. В ходе морфологического анализа `natasha` опирается на библиотеку `pymorphy2`, которая для каждого слова выдает все возможные теги, `natasha` же осуществляет разрешение неоднозначностей.

Сначала воспользуемся `pymorphy2` для получения всех возможных тегов для слова "мыла":

In [10]:
morph = pymorphy2.MorphAnalyzer()
morph.parse('мыла')

[Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut sing,gent'), normal_form='мыло', score=0.333333, methods_stack=((DictionaryAnalyzer(), 'мыла', 54, 1),)),
 Parse(word='мыла', tag=OpencorporaTag('VERB,impf,tran femn,sing,past,indc'), normal_form='мыть', score=0.333333, methods_stack=((DictionaryAnalyzer(), 'мыла', 2074, 8),)),
 Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut plur,nomn'), normal_form='мыло', score=0.166666, methods_stack=((DictionaryAnalyzer(), 'мыла', 54, 6),)),
 Parse(word='мыла', tag=OpencorporaTag('NOUN,inan,neut plur,accs'), normal_form='мыло', score=0.166666, methods_stack=((DictionaryAnalyzer(), 'мыла', 54, 9),))]

Действительно, это может быть и существительное "мыло" в родительном падеже (а также в двух других падежных формах), и глагол "мыть" в женском роде прошедшем времени. Можно обратить внимание, что для каждого варианта тега `pymorphy2` возвращает `score` - по сути, это нормализованная (между разными тегами) частота того, как часто используется именно эта форма в части OpenCorpora со снятой частеречной омонимией. 

Попробуем обработать ту же фразу с помощью `natasha`:

In [11]:
ambiguity_example = natasha.Doc('Мама мыла раму.')
ambiguity_example.segment(segmenter)

# Модель, обеспечивающая, в том числе, разрешение
# частеречной омонимии
emb = natasha.NewsEmbedding()
morph_tagger = natasha.NewsMorphTagger(emb)

ambiguity_example.tag_morph(morph_tagger)

In [12]:
ambiguity_example.tokens

[DocToken(stop=4, text='Мама', pos='NOUN', feats=<Anim,Nom,Fem,Sing>),
 DocToken(start=5, stop=9, text='мыла', pos='VERB', feats=<Imp,Fem,Ind,Sing,Past,Fin,Act>),
 DocToken(start=10, stop=14, text='раму', pos='NOUN', feats=<Inan,Acc,Fem,Sing>),
 DocToken(start=14, stop=15, text='.', pos='PUNCT')]

В результате применения метода `tag_morph()` каждый токен был снабжен морфологической информацией - поле `pos` (part-of-speech) содержит часть речи, к которой был отнесен токен, а `feats` - набор граммем, характеризующих форму слова. При этом - обратите внимание! - тег для каждого слова только один, `natasha` провела разрешение противоречий.

Проставим тэги для первой новости:

In [13]:
doc.tag_morph(morph_tagger)

In [14]:
doc.tokens[:5]

[DocToken(stop=1, text='С', pos='ADP'),
 DocToken(start=2, stop=3, text='1', pos='ADJ'),
 DocToken(start=4, stop=10, text='января', pos='NOUN', feats=<Inan,Gen,Masc,Sing>),
 DocToken(start=11, stop=15, text='2000', pos='ADJ'),
 DocToken(start=16, stop=20, text='года', pos='NOUN', feats=<Inan,Gen,Masc,Sing>)]

### Лемматизация

Лемматизация - приведение слова к начальной форме. Для этого, очевидно, частеречная омонимия должна быть уже разрешена, соответственно, лемматизация опирается на результат морфологического анализа.

In [15]:
morph_vocab = natasha.MorphVocab()

Лемматизация применяется не ко всему документу, а к заданным токенам:

In [16]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

In [17]:
doc.tokens[:5]

[DocToken(stop=1, text='С', pos='ADP', lemma='с'),
 DocToken(start=2, stop=3, text='1', pos='ADJ', lemma='1'),
 DocToken(start=4, stop=10, text='января', pos='NOUN', feats=<Inan,Gen,Masc,Sing>, lemma='январь'),
 DocToken(start=11, stop=15, text='2000', pos='ADJ', lemma='2000'),
 DocToken(start=16, stop=20, text='года', pos='NOUN', feats=<Inan,Gen,Masc,Sing>, lemma='год')]

В результате токены снабжаются дополнительным свойством - `lemma`, содержащим начальную форму слова.

## 1.3 Синтаксический разбор

В ходе синтаксического разбора строится дерево зависимостей между словами в каждом предложении. В библиотеке `natasha` синтаксический разбор реализуется классом `NewsSyntaxParser`. Экземпляр этого класса применяется к документу. При этом документ должен быть сегментирован (разбит на токены), но морфологический анализ проводить не обязательно (компонент синтаксического разбора не опирается на морфологические тэги и леммы).

In [18]:
syntax_parser = natasha.NewsSyntaxParser(emb)
doc.parse_syntax(syntax_parser)

Отобразить дерево зависимостей графически можно следующим образом:

In [19]:
doc.sents[0].syntax.print()

        ┌► С                case
  ┌►┌─┌─└─ 1                obl
  │ │ └──► января           flat
  │ │   ┌► 2000             amod
  │ └──►└─ года             nmod
  │     ┌► все              det
  │   ┌►└─ телеканалы       nsubj
  │   │ ┌► будут            aux
┌─└───└─└─ оплачивать       
│     └►┌─ услуги           obj
│     ┌─└► передачи         nmod
│   ┌─└──► телерадиосигнала iobj
│   │ ┌──► по               case
│   │ │ ┌► единому          amod
│   └►└─└─ тарифу           obl
└────────► .                punct


Однако, используя свойство `syntax`, можно написать код, анализирующий зависимости, и, например, извлекающий основу предложения (подлежащее и сказуемое):

In [20]:
def extract_basis(syntactic_tree):
    # id2token = {x.id: x for x in syntactic_tree.tokens}
    # Найти главное слово - сказуемое
    root = [x for x in syntactic_tree.tokens if x.rel == 'root'][0]
    # Проверить, есть ли у него какие-то модификаторы (aux)
    aux = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel=='aux' or x.rel=='aux:pass')]
    # Найти подлежащее
    subject = [x for x in syntactic_tree.tokens if x.head_id == root.id and (x.rel == 'nsubj' or x.rel=='nsubj:pass')]
    return subject[0].text, aux[0].text + ' ' + root.text if aux else root.text  

for sent in doc.sents:
    print(extract_basis(sent))

('телеканалы', 'будут оплачивать')
('постановление', 'является')
('телеканалы', 'оплачивают')
('тарифы', 'будут увеличены')


Конечно, эта функция далека от той, что можно использовать на практике (например, она совершенно не учитывает сложных предложений, у которых может быть несколько основ), но общие идеи можно использовать.

## 1.4 Именованные сущности

К именованным сущностям относятся имена людей, географических объектов, компаний и т.д. 

TODO

# 2. Векторная модель документа, мешок слов и классификация

Построим простой тематический классификатор новостей, основанный на векторной модели документа. Для этого нужно:

1. Выделить множество признаков, которое будет соответствовать множеству всех слов, которые встречаются во всех новостных заметках (возможно, исключив самые частотные и самые редко встречающиеся).
2. Преобразовать каждую новостную заметку в вектор, в котором ненулевые значения будут соответствовать признакам-словам, которые присутствуют в заметке. Само ненулевое значение может быть получено по-разному - частота слова, TF-IDF, и пр.
3. Определить целевую переменную (метку классификации).
4. Обучить модель (логистической регрессии).

Все эти шаги легко проделать с помощью библиотеки `scikit-learn`. Так, для первых двух этапов в ней предусмотрен набор "векторизаторов" (`CountVectorizer`, `TfIdfVectorizer`), а для обучения логистической регрессии `linear_model.LogisticRegression`. Кроме того, библиотека предоставляет набор инструментов, облегчающих типовые задачи машинного обучения: разбиение на обучающую и тестовую выборки, кросс-валидация и пр.

In [21]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score

Поставим задачу классификации как классификация новостей, относящихся к теме "Культура". Соответственно, уберем из набора данных все лишние поля и сформируем целевую метку:

In [22]:
df['target'] = (df.topic == 'Культура').astype(np.int8)
df = df[['text', 'target']]

In [23]:
df.dropna(inplace=True)

C:\Users\hatter\AppData\Local\Temp\ipykernel_18612\1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [24]:
df

,text,target
0,С 1 января 2000 года все телеканалы будут опла...,0
1,Германский автопромышленный концерн Volkswagen...,0
2,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",0
3,Две крупнейших телекоммуникационных компании С...,0
4,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",0
...,...,...
197733,Российский боец смешанного стиля (MMA) Алексей...,0
197734,Американский боец смешанного стиля (MMA) Фрэн...,0
197735,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,0
197736,"Президент России Владимир Путин, выступая на з...",1


In [25]:
vectorizer = CountVectorizer(
    max_df=0.7,    # Делать признаками слова, которые содержатся в не более, чем заданной доле документов
    min_df=10      # Делать признаки из слов, которые содержатся, по крайней мере, в заданном количестве документов
)

In [26]:
X = vectorizer.fit_transform(df.text)

In [27]:
X

<197737x106721 sparse matrix of type '<class 'numpy.int64'>'
	with 24058646 stored elements in Compressed Sparse Row format>

Полученная матрица является разреженной матрицей. Видно, что в результате векторизации был сделан 106721 признак (каждый соответствует слову). Если бы матрица не была разреженной, то в ней было бы больше ста миллиардов значений (что, конечно, не поместилось бы в память), однако ненулевых значений в ней около 24 млн., и это вполне разумное количество.

Для обучения логистической регрессии (да и большиства моделей) следует нормализовать данные. Для этого в `sklearn` есть несколько методов. Самый популярный выбор - `StandardScaler`, который пытается привести распределение каждого атрибута к центрированному в 0 нормальному, для этого из значения атрибута вычитается среднее арифметическое по столбцу и делится на стандартное отклонение. Но мы поступим немного по-другому. Нормализуем каждую строку, перейдя к относительной частоте слова. Для этого разделим каждую строку матрицы на сумму соответствующей строки (применив так называемое сглаживание Лапласа, чтобы избежать деления на 0):

In [28]:
X = X / (X.sum(axis=1) + 1)

Разделим имеющиеся данные на обучающее и тестовое множества:

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, df.target, test_size=0.2, random_state=42, stratify=df.target)

Обучим модель логистической регрессии:

In [30]:
lm = LogisticRegression()
lm.fit(X_train, y_train)

LogisticRegression()

Наконец, оценим качество модели:

In [31]:
print('Accuracy:', accuracy_score(y_test, lm.predict(X_test)))
print('ROC_AUC:', roc_auc_score(y_test, lm.predict_proba(X_test)[:, 1]))

Accuracy: 0.9419439668251239
ROC_AUC: 0.9939112907002375


Получилось вполне неплохо! Такое высокое качество объясняется несколькими причинами:

1. Всё-таки, это очень простая задача. Во-первых, мы довольно сильно ограничили количество классов как в обучающей, так и в тестовой выборках. Во-вторых, тематическая классификация в постановке "Культура"-против-остальных действительно довольно неплохо может решаться просто по вхождению некоторых слов: "балет", "концерт" и пр.
2. Некоторые шаги выше для простоты были сделаны не совсем "честно". Например, то, что векторизатор обучался на всём множестве документов (включая тестовое множество). Очевидно, это не совсем соответствует сценарию реального применения, когда векторизатор обучается только на обучающем множестве, а вновь поступающие документы просто им обрабатываются и новая лексика, которая не была известна на момент обучения модели, будет просто проигнорирована.

Кроме того, есть еще ряд путей для возможного улучшения:

1. Более надежная оценка модели достигается с применением кросс-валидации.
2. Можно попробовать модифицировать процесс выделения признаков, добавив лемматизацию и более точную токенизацию, используя библиотеку `natasha`.
3. Можно попробовать получать признаки, взвешенные по TF-IDF.

In [32]:
df = df.sample(frac=1.).reset_index(drop=True)
df_test = df[:1000].copy()
df_train = df[1000:].copy().reset_index(drop=True)
del df

In [33]:
vectorizer = CountVectorizer(
    max_df=0.7,    # Делать признаками слова, которые содержатся в не более, чем заданной доле документов
    min_df=10      # Делать признаки из слов, которые содержатся, по крайней мере, в заданном количестве документов
)
X_train = vectorizer.fit_transform(df_train.text)
X_train = X_train / (X_train.sum(axis=1) + 1)
lm = LogisticRegression()
lm.fit(X_train, df_train.target)

LogisticRegression()

In [34]:
X_test = vectorizer.transform(df_test.text)
X_test = X_test / (X_test.sum(axis=1) + 1)
print('Accuracy:', accuracy_score(df_test.target, lm.predict(X_test)))
print('ROC_AUC:', roc_auc_score(df_test.target, lm.predict_proba(X_test)[:, 1]))

Accuracy: 0.943
ROC_AUC: 0.9963486388763165


Попробуем обучить модель с более "интеллектуальным" подходом к подготовке признаков - приведением к нормальной форме (лемме) и выбрасыванием всех слов, кроме существительных и глаголов. К сожалению, морфологический разбор требует времени, поэтому применение подобных преобразований ко всему набору данных оказывается довольно длительным процессом (на практике использовать можно, но в целях обучения - нецелесообразно). Поэтому сократим обучающую часть - будем использовать для обучения лишь 10 тыс. сообщений:

In [35]:
class SmartTokenizer:
    """Класс для выделения токенов.
    
    Использует библиотеку natasha для токенизации и лемматизации,
    оставляет только существительные и глаголы."""
    
    def __init__(self):
        emb = natasha.NewsEmbedding()
        self.segmenter = natasha.Segmenter()
        self.morph_tagger = natasha.NewsMorphTagger(emb)
        self.morph_vocab = natasha.MorphVocab()
    
    def __call__(self, text):
        doc = natasha.Doc(text)
        doc.segment(self.segmenter)
        doc.tag_morph(self.morph_tagger)
        tokens = []
        for token in doc.tokens:
            if token.pos in ['NOUN', 'VERB']:
                token.lemmatize(morph_vocab)
                tokens.append(token.lemma)
        return tokens

improved_vectorizer = CountVectorizer(
    tokenizer=SmartTokenizer(),
    token_pattern=None,
    max_df=0.7,    # Делать признаками слова, которые содержатся в не более, чем заданной доле документов
    min_df=10      # Делать признаки из слов, которые содержатся, по крайней мере, в заданном количестве документов
)

# Подмножество обучающего набора
df_train_small = df_train[:10000]

X_train = improved_vectorizer.fit_transform(df_train_small.text)
X_train = X_train / (X_train.sum(axis=1) + 1)
lm = LogisticRegression()
lm.fit(X_train, df_train_small.target)

LogisticRegression()

Несмотря на то, что мы ограничили обучающее множество всего 10 тыс. новостных сообщений, ячейка выполнялась существенно дольше, чем подготовка почти 200 тыс. сообщений с помощью простой токенизации (около 3 минут на моем компьютере).

In [36]:
X_test = improved_vectorizer.transform(df_test.text)
X_test = X_test / (X_test.sum(axis=1) + 1)
print('Accuracy:', accuracy_score(df_test.target, lm.predict(X_test)))
print('ROC_AUC:', roc_auc_score(df_test.target, lm.predict_proba(X_test)[:, 1]))

Accuracy: 0.863
ROC_AUC: 0.9975894914030279


Результат довольно противоречивый. Во-первых, разные метрики по-разному "отреагировали" на изменение. Точность (accuracy) существенно уменьшилась, однако ROC AUC почти не изменился. По-прежнему высокий ROC AUC говорит о том, что модель неплохо ранжирует результаты (то есть, выдает, как правило, большие значения вероятности для новостных сообщений, действительно относящихся к теме Культура). Низкая же точность свидетельствует о том, что модель стала хуже калиброванной. То есть, порог 0.5, который используется по умолчанию, чтобы переводить вероятности, появляющиеся на выходе модели, в нули и единицы, не очень хорош. Попробуем другие пороги:

In [37]:
for thr in [0.2, 0.3, 0.4, 0.5, 0.6]:
    print(f'Accuracy (thr={thr}): {accuracy_score(df_test.target, lm.predict_proba(X_test)[:, 1] > thr):.4f}')    

Accuracy (thr=0.2): 0.7580
Accuracy (thr=0.3): 0.9830
Accuracy (thr=0.4): 0.9410
Accuracy (thr=0.5): 0.8630
Accuracy (thr=0.6): 0.8140


Видно, что при пороге классификации около 0.3 точность достигает аж 0.97! Правда, идея использования тестового множества для подбора порога - так себе. Посмотрим что с обучающим множеством:

In [38]:
for thr in [0.2, 0.3, 0.4, 0.5, 0.6]:
    print(f'Accuracy (thr={thr}): {accuracy_score(df_train_small.target, lm.predict_proba(X_train)[:, 1] > thr):.4f}')    

Accuracy (thr=0.2): 0.7516
Accuracy (thr=0.3): 0.9781
Accuracy (thr=0.4): 0.9456
Accuracy (thr=0.5): 0.8661
Accuracy (thr=0.6): 0.8007


Тот же эффект. Таким образом, мы можем обоснованно выбрать порог классификации в 0.3 на обучающем множестве и добиться достаточно высокой точности.

In [39]:
X_train

<10000x6402 sparse matrix of type '<class 'numpy.float64'>'
	with 605042 stored elements in COOrdinate format>

Обратите внимание, что из-за применения лемматизации матрица признаков оказалась существенно  менее широкой и гораздо более "плотной". Кроме того, для обучения достаточно хорошего классификатора хватило всего 10 тыс. сообщений.

## Интерпретация модели

Одним из достоинств линейных моделей является их интерпретируемость. Действительно, математически наша модель записывается в виде:

$$
y=\sigma(w_1x_1 + ... w_nx_n)
$$

Где $x_1$ - это частота слова в документе. Соответственно, положительные коэффициенты $w_1$ соответствуют тем словам, присутствие которых повышает вероятность положительной классификации объекта, а отрицательные - тем, которые понижают.


In [40]:
features = [''] * len(improved_vectorizer.vocabulary_)
for k, v in improved_vectorizer.vocabulary_.items():
    features[v] = k

In [41]:
fdf = pd.DataFrame({'word': features,
                    'effect': lm.coef_.ravel()})

In [42]:
fdf.sort_values(['effect'], ascending=False)[:20]

,word,effect
5916,фильм,14.210825
1955,картина,8.264702
4578,режиссер,5.570922
140,альбом,5.291805
4694,роль,5.204574
100,актер,5.174695
1071,группа,5.003640
5450,театр,4.592963
4055,премия,4.356246
2702,музей,4.014702


# 3 Эмбеддинги слов (вложения, embeddings)

В библиотеке `natasha` есть также набор эмбеддингов слов для русского языка. На самом деле, мы их уже раньше использовали (не напрямую, но передавали в качестве параметров классам, осуществляющим разные операции лингвистического анализа). Эмбеддинги слов хранятся в экземпляре `emb` класса `natasha.NewsEmbedding`. Простейший способ получения эмбеддинга слова: `emb['слово']`:

In [43]:
emb['слово']

array([ 2.55429476e-01,  5.89054311e-03,  1.57977819e-01,  1.82602152e-01,
       -2.25324184e-01, -2.96944350e-01,  3.48798364e-01, -6.22543991e-02,
       -4.75153625e-01, -4.50215518e-01,  6.82648644e-02, -1.88942790e-01,
       -2.57037114e-02,  3.50426584e-01, -3.09493065e-01,  3.69875431e-01,
       -3.96766871e-01,  3.58251959e-01, -1.06716901e-01,  2.17676908e-01,
       -8.67002904e-02, -1.70834616e-01, -6.49302825e-02, -4.35781889e-02,
       -1.85484979e-02, -6.33450031e-01,  3.89721841e-01, -2.16791287e-01,
       -4.60259497e-01,  2.69960612e-01, -4.14235801e-01, -1.01559913e+00,
        3.47699993e-03,  3.28725785e-01,  1.26028627e-01,  7.60548934e-02,
        3.50357533e-01, -5.79251111e-01,  9.40726846e-02, -1.22430496e-01,
       -2.66330205e-02,  7.68245697e-01, -3.23462449e-02, -3.32074314e-01,
       -1.42027378e-01, -3.43501210e-01, -3.53978842e-01,  3.05407830e-02,
        3.05212557e-01, -1.34403229e-01,  6.84357047e-01, -3.43205690e-01,
        3.83923985e-02, -

Попробуем использовать эмбеддинги для поиска близких по значению слов (нужно помнить, что это не синонимы в полной мере) и для рассуждения по аналогии. Для этого реализуем пару функций, осуществляющих поиск слов, ближайших к заданному вектору: 

In [44]:
import heapq

def squared_euclidean_sim(x, y):
    return -np.sum((x - y) ** 2)

def find_closest_word(emb, v, similarity=squared_euclidean_sim):
    """Поиск одного ближайшего слова."""
    closest_word_value = -1
    closest_word = None
    for word in emb.vocab.words:
        sim = similarity(emb[word], v)
        if sim > closest_word_value:
            closest_word_value = sim
            closest_word = word
    return closest_word

def find_closest_words(emb, v, k=10, similarity=squared_euclidean_sim):
    """Поиск k ближайших слов.
    
    Примечание: функция не очень эффективна, исключительно для демонстрации.
    """
    dists = [(-similarity(emb[word], v), word) for word in emb.vocab.words]
    heapq.heapify(dists)
    result = []
    for i in range(k):
        top = heapq.heappop(dists)
        result.append(top[1])
    return result

find_closest_words(emb, emb['море'], 10)

['море',
 'средиземном',
 'моря',
 'черном',
 'черное',
 'баренцевом',
 'берегов',
 'средиземное',
 'эгейском',
 'южно-китайское']

Попробуем т.н. "рассуждения по аналогии". Учитывая специфику набора данных, на которых были обучены вложения слов, пробовать аналогии вроде "царь - мужчина + женщина ~ царица" довольно бессмыссленно. Но вот со столицами и странами должно более или менее работать: 

In [45]:
find_closest_words(emb, emb['франция'], 10)

['франция',
 'германия',
 'бельгия',
 'италия',
 'великобритания',
 'париж',
 'австрия',
 'испания',
 'швеция',
 'швейцария']

In [46]:
find_closest_words(emb, (emb['берлин'] - emb['германия']) + emb['франция'], 5)

['париж', 'берлин', 'франция', 'брюссель', 'лондон']

In [47]:
find_closest_words(emb, (emb['рим'] - emb['италия']) + emb['германия'], 5)

['берлин', 'рим', 'мюнхен', 'германия', 'гамбург']

Действительно, в какой-то мере работает.

Библиотека `natasha` предоставляет специальный класс `NavecEmbedding`, совместимый с фреймворком PyTorch (т.е., реализующий `torch.nn.Module`), и предназначенный для перевода номеров слов в вектора эмбеддингов в рамках нейронных сетей, созданных с помощью этого фреймворка.

In [48]:
import torch

from slovnet.model.emb import NavecEmbedding

In [49]:
emb_layer = NavecEmbedding(emb)

C:\opt\miniconda3\envs\aim\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:212.)
  torch.from_numpy(navec.pq.indexes),


Стандартный подход к обработке текста с помощью нейронных сетей заключается в том, что всем (или почти) словам присваиваются номера, соответствующие строчкам в таблице эмбеддингов. Слой эмбеддингов получает на вход вектор таких идентификаторов слов и на выходе формирует матрицу, составленную из эмбеддиннгов соответствующих слов:

In [50]:
input = torch.tensor([1, 2, 3])
emb_layer(input).shape

torch.Size([3, 300])

Полученный тензор имеет размерность $3 \times 300$ - это три значения эмбеддингов длиной 300, соответствующие трём входным словам. 

Как правило, матрица эмбеддингов содержит еще два специальных значения:

- значение для неизвестного слова. Действительно, входное слово может быть редким, может быть написано с опечаткой, поэтому для него может просто не быть эмбеддинга. Все такие слова переводятся в специальное слово `<unk>` и используют одинаковое значение эмбеддинга;
- как правило, сеть обрабатывает данные минибатчами. Все последовательности одного минибатча должны быть одинаковой длины (но в разных минибатчах теоретически могут использоваться разные длины), но фактически длина текста может быть различной, поэтому если текст короче, чем длина последовательности минибатча, то он дополняется специальным токеном `<pad>`.

Сами значения для неизвестного токена и токена выравнивания (они полезны для преобразования текста в набор идентификаторов), могут быть получены следующим образом:

In [51]:
emb.vocab.unk_id

250000

In [52]:
emb.vocab.pad_id

250001

Номера прочих токенов:

In [53]:
emb.vocab['море']

117429

Используя эти данные можно написать функцию, осуществляющую преобразование текста в номера токенов, поддерживаемых слоем эмбеддингов:

In [54]:
def text_to_ids(emb, text: str, length: int) -> torch.tensor:
    """Преобразование строки в тензор с номерами токенов."""
    # Пунктуационные токены (их кодировать не будем)
    punct = [',', '.', ';', ':', '-', '...', '!', '?']
    # Проведем токенизацию текста
    d = natasha.Doc(text)
    d.segment(segmenter)
    # Для каждого токена, который найдется в словаре эмбеддингов подставим
    # его номер, для прочих подставим номер <unk>
    tmp = torch.tensor([emb.vocab.get(x.text.lower(), emb.vocab.unk_id)
                       for x in d.tokens
                       if x.text not in punct][:length])
    # Дополним последовательность (спереди) токенами <pad>
    return torch.nn.functional.pad(tmp, (length - len(tmp), 0), "constant", emb.vocab.pad_id)

In [55]:
text_to_ids(emb, 'Я помню чюдное мгновенье!', 10)

tensor([250001, 250001, 250001, 250001, 250001, 250001, 248820, 162667, 250000,
        111457])

Видно, что все слова были представлены были представлены номерами токенов, слева последовательность дополнена до длины 10 токенами выравнивания (250001), а слово с ошибкой ("чюдное") не было найдено в словаре, поэтому для него используется идентификатор неизвестного токена (250000).

# 4 Обучение нейронной сети (LSTM)

In [56]:
import torch.nn as nn

Преобразуем обучающее и тестовое множества в наборы токенов:

In [57]:
X = torch.stack([text_to_ids(emb, x, 50) for x in df_train_small.text], 0)
y = torch.unsqueeze(torch.tensor(df_train_small.target, dtype=torch.float32), 1)

X_test = torch.stack([text_to_ids(emb, x, 50) for x in df_test.text], 0)
y_test = torch.unsqueeze(torch.tensor(df_test.target, dtype=torch.float32), 1)

In [58]:
X[0], y[0]

(tensor([ 33712, 209045, 212839, 119768, 178312, 155642,  59225, 108254,  60824,
          70676, 142929, 173082, 205454, 119768, 193013,   1111,   8188,  33712,
         178312, 171949, 229673,  42440,  69009,  78526, 240054, 118515, 193605,
         115595,  79270, 130886, 250000, 104165, 193522, 209969, 138046, 250000,
          35088, 208886, 250000,  75189, 221132, 239317, 102842, 175751, 137130,
          16047,  13531,  22480, 129425,  40049]),
 tensor([1.]))

Создадим загрузчики. В данном случае, все данные помещаются в память и хранятся в одном тензоре. Для подобных случаев в PyTorch есть специальный вид `Dataset` - `TensorDataset`:

In [59]:
data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=8)
test_data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

In [60]:
class SimpleLSTMClassifier(nn.Module):
    """Простой классификатор текста."""
    
    def __init__(self):
        super().__init__()
        # Специальная обёртка, позволяющая использовать
        # эмбеддинги natasha как слой сети PyTorch
        self.embedding = NavecEmbedding(emb)
        # LSTM-слой.
        # Обратите внимание на batch_first=True !
        # По умолчанию этот параметр равен False и первая размерность
        # интерпретируется как длина последовательности, а не батча - 
        # если это не поменять, то сеть будет учиться на "каше" из данных
        self.lstm = nn.LSTM(300,   # размерность элемента последовательности (эмбеддинга)
                            30,    # выходная размерность
                            batch_first=True)
        self.fc = nn.Linear(30, 1)
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        # Берем только последний выход LSTM ячейки
        # (см. вид архитектуры много-к-одному)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [61]:
model = SimpleLSTMClassifier()

In [62]:
model(torch.unsqueeze(text_to_ids(emb, 'Утро туманное...', 5), 0))

tensor([[-0.0269]], grad_fn=<AddmmBackward0>)

In [63]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

for epoch in range(5):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()
        pred_logits = model(X)
        loss = criterion(pred_logits, y)
        loss.backward()
        optimizer.step()
    model.eval()
    preds = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds.append(torch.sigmoid(pred_logits))
    preds = torch.cat(preds)
    print(f'Epoch {epoch}: train loss={loss.detach().item():.4f}. ' \
          f'Test accuracy {accuracy_score(df_test.target, preds.numpy() > 0.5):.4f} ' \
          f'ROC_AUC {roc_auc_score(df_test.target, preds.numpy()):.4f}')

Epoch 0: train loss=0.0247. Test accuracy 0.9790 ROC_AUC 0.9915
Epoch 1: train loss=0.0071. Test accuracy 0.9840 ROC_AUC 0.9953
Epoch 2: train loss=0.0066. Test accuracy 0.9870 ROC_AUC 0.9941
Epoch 3: train loss=0.0029. Test accuracy 0.9900 ROC_AUC 0.9964
Epoch 4: train loss=0.0018. Test accuracy 0.9840 ROC_AUC 0.9977
